<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/transformer/SelfAttention_in_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 第2課題: Transformerの心臓部「Self-Attention (QKV)」の解剖

第1課題では、GPTがどのようにテキストを生成するかの全体像を学びました。
1.  **単語埋め込み**（単語をベクトル化）
2.  **Attention**（文脈に応じてベクトルを変換）
3.  **LM Head**（次の単語を予測）

しかし、第1課題では `Attention` が「なぜ、どのように」文脈を理解できるのか、その**計算の核心部分**（QKV）を意図的に省略しました。

この第2課題では、単語の「順序」を扱う `Positional Encoding` と、`Transformer` の核である `Self-Attention` の仕組み（QKV, Multi-Head, FFN）について、ステップバイステップで解剖していきます。

In [95]:
#@title 模式図を表示
from IPython.display import display, HTML

svg_code_string = """<svg width="1200px" viewBox="0 0 1200 500" xmlns="http://www.w3.org/2000/svg">
  <defs>
    <linearGradient id="inputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5BA3F5;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="queryGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#E74C3C;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#EC7063;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="keyGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#F39C12;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F5B041;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="valueGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#9B59B6;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#BB8FCE;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="attnGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#16A085;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#48C9B0;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="outputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#52BE80;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="ffnGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#2E86C1;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5DADE2;stop-opacity:1" />
    </linearGradient>

    <marker id="arrowhead" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#555" />
    </marker>

    <marker id="arrowheadRed" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#E74C3C" />
    </marker>

    <marker id="arrowheadOrange" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#F39C12" />
    </marker>

    <marker id="arrowheadPurple" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#9B59B6" />
    </marker>
  </defs>

  <text x="600" y="25" font-size="18" font-weight="bold" text-anchor="middle" fill="#333">
    Self-Attention Mechanism (Query, Key, Value)
  </text>

  <g id="step_labels">
    <text x="50" y="60" font-size="14" font-weight="bold" fill="#666">Step 1:</text>
    <text x="50" y="75" font-size="13" fill="#666">Embed + PE</text>

    <text x="370" y="60" font-size="14" font-weight="bold" fill="#666">Step 2: QKV</text>

    <text x="555" y="60" font-size="14" font-weight="bold" fill="#666">Step 3: Attention</text>

    <text x="800" y="60" font-size="14" font-weight="bold" fill="#666">Step 4: FFN</text>
  </g>

  <g id="text_input" transform="translate(30, 200)">
    <rect x="0" y="0" width="60" height="200" fill="#F8F9FA" stroke="#CCC" stroke-width="1" rx="3"/>
    <text x="30" y="20" font-size="13" font-weight="bold" text-anchor="middle" fill="#333">Text</text>

    <text x="30" y="70" font-size="12" text-anchor="middle" fill="#555">"man</text>
    <text x="30" y="120" font-size="12" text-anchor="middle" fill="#555">bites</text>
    <text x="30" y="170" font-size="12" text-anchor="middle" fill="#555">dog"</text>
  </g>

  <path d="M 95 300 L 115 300" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="embedding" transform="translate(120, 200)">
    <rect x="0" y="0" width="70" height="200" fill="#8E44AD" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Word</text>
    <text x="35" y="38" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Embed</text>

    <line x1="10" y1="70" x2="60" y2="70" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="120" x2="60" y2="120" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="170" x2="60" y2="170" stroke="white" stroke-width="1.5" opacity="0.6"/>
  </g>

  <g id="positional_encoding" transform="translate(120, 110)">
    <rect x="0" y="0" width="70" height="60" fill="#E67E22" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Positional</text>
    <text x="35" y="38" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Encoding</text>
    <text x="35" y="55" font-size="10" text-anchor="middle" fill="white">(PE)</text>
  </g>

  <path d="M 155 175 L 155 195" stroke="#E67E22" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <circle cx="155" cy="185" r="12" fill="white" stroke="#333" stroke-width="1.5"/>
  <text x="155" y="191" font-size="16" font-weight="bold" text-anchor="middle" fill="#333">+</text>

  <path d="M 195 300 L 215 300" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="input_vec" transform="translate(220, 200)">
    <rect x="0" y="0" width="70" height="200" fill="url(#inputGrad)" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Input</text>
    <text x="35" y="38" font-size="11" text-anchor="middle" fill="white">Vec</text>

    <line x1="10" y1="70" x2="60" y2="70" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="120" x2="60" y2="120" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="170" x2="60" y2="170" stroke="white" stroke-width="1.5" opacity="0.6"/>
  </g>

  <rect x="305" y="130" width="590" height="340" fill="none" stroke="#2C3E50"
        stroke-width="3" rx="8" stroke-dasharray="8,4" opacity="0.8"/>
  <text x="640" y="125" font-size="14" font-weight="bold" text-anchor="middle" fill="#2C3E50">
    Transformer Block
  </text>

  <path d="M 295 300 L 330 300" stroke="#555" stroke-width="2" fill="none"/>

  <g id="qkv_split">
    <circle cx="335" cy="300" r="4" fill="#333"/>

    <path d="M 335 300 L 370 210" stroke="#E74C3C" stroke-width="2" fill="none" marker-end="url(#arrowheadRed)"/>
    <path d="M 335 300 L 370 300" stroke="#F39C12" stroke-width="2" fill="none" marker-end="url(#arrowheadOrange)"/>
    <path d="M 335 300 L 370 390" stroke="#9B59B6" stroke-width="2" fill="none" marker-end="url(#arrowheadPurple)"/>
  </g>

  <g id="query" transform="translate(375, 160)">
    <rect x="0" y="0" width="75" height="80" fill="url(#queryGrad)" stroke="#E74C3C" stroke-width="2" rx="3"/>
    <text x="37.5" y="25" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Query</text>
    <text x="37.5" y="45" font-size="10" text-anchor="middle" fill="white">(Q = W<tspan baseline-shift="sub" font-size="70%">q</tspan>X)</text>

    <line x1="10" y1="60" x2="65" y2="60" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>

  <g id="key" transform="translate(375, 260)">
    <rect x="0" y="0" width="75" height="80" fill="url(#keyGrad)" stroke="#F39C12" stroke-width="2" rx="3"/>
    <text x="37.5" y="25" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Key</text>
    <text x="37.5" y="45" font-size="10" text-anchor="middle" fill="white">(K = W<tspan baseline-shift="sub" font-size="70%">k</tspan>X)</text>

    <line x1="10" y1="60" x2="65" y2="60" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>

  <g id="value" transform="translate(375, 360)">
    <rect x="0" y="0" width="75" height="80" fill="url(#valueGrad)" stroke="#9B59B6" stroke-width="2" rx="3"/>
    <text x="37.5" y="25" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Value</text>
    <text x="37.5" y="45" font-size="10" text-anchor="middle" fill="white">(V = W<tspan baseline-shift="sub" font-size="70%">v</tspan>X)</text>

    <line x1="10" y1="60" x2="65" y2="60" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>

  <g id="attention_weights" transform="translate(500, 210)">
    <rect x="0" y="0" width="110" height="80" fill="url(#attnGrad)" stroke="#16A085" stroke-width="2" rx="3"/>
    <text x="55" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Attention</text>
    <text x="55" y="38" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Weights</text>
    <text x="55" y="55" font-size="10" text-anchor="middle" fill="white">Softmax(Q K<tspan baseline-shift="super" font-size="70%">T</tspan></text>
    <text x="55" y="70" font-size="10" text-anchor="middle" fill="white">/√d<tspan baseline-shift="sub" font-size="70%">k</tspan>)</text>
  </g>

  <path d="M 455 200 L 495 240" stroke="#E74C3C" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <path d="M 455 300 L 495 260" stroke="#F39C12" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="weighted_sum" transform="translate(530, 350)">
    <ellipse cx="30" cy="40" rx="30" ry="30" fill="#F0F0F0" stroke="#333" stroke-width="2"/>
    <text x="30" y="37" font-size="20" font-weight="bold" text-anchor="middle" fill="#333">×</text>
    <text x="30" y="50" font-size="9" text-anchor="middle" fill="#555">
      <tspan x="30" dy="0">weighted</tspan>
      <tspan x="30" dy="10">sum</tspan>
    </text>
  </g>

  <path d="M 555 295 L 555 345" stroke="#16A085" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <path d="M 455 400 L 525 380" stroke="#9B59B6" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="attention_output" transform="translate(670, 290)">
    <rect x="0" y="0" width="80" height="80" fill="url(#outputGrad)" stroke="#27AE60" stroke-width="2" rx="3"/>
    <text x="40" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Attention</text>
    <text x="40" y="38" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Output</text>
    <text x="40" y="55" font-size="9" text-anchor="middle" fill="white">(context vec)</text>

    <line x1="10" y1="70" x2="70" y2="70" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>
  <path d="M 590 380 L 665 330" stroke="#27AE60" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <path d="M 755 330 L 785 330" stroke="#27AE60" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="ffn" transform="translate(790, 290)">
    <rect x="0" y="0" width="90" height="80" fill="url(#ffnGrad)" stroke="#2E86C1" stroke-width="2" rx="3"/>
    <text x="45" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Feed Forward</text>
    <text x="45" y="38" font-size="12" font-weight="bold" text-anchor="middle" fill="white">Network</text>
    <text x="45" y="55" font-size="9" text-anchor="middle" fill="white">(2-layer MLP)</text>

    <line x1="12" y1="70" x2="78" y2="70" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>

  <path d="M 885 330 L 925 330" stroke="#27AE60" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <text x="940" y="335" font-size="20" font-weight="bold" text-anchor="middle" fill="#666">...</text>

  <path d="M 955 330 L 995 330" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="lm_head" transform="translate(1000, 290)">
    <rect x="0" y="0" width="80" height="80" fill="#E74C3C" stroke="#333" stroke-width="2" rx="3"/>
    <text x="40" y="30" font-size="12" font-weight="bold" text-anchor="middle" fill="white">LM Head</text>
    <text x="40" y="48" font-size="9" text-anchor="middle" fill="white">(next word</text>
    <text x="40" y="61" font-size="9" text-anchor="middle" fill="white">prediction)</text>
  </g>
  </svg>
"""

# HTML() を使って表示します
display(HTML(svg_code_string))

In [ ]:
#@title 【まず実行】必要なライブラリのインストールと設定
# Transformers ライブラリ（GPT-2モデルを使うため）
# tqdm は進捗バーを表示するため
# (GPUは不要です)
!pip install -q transformers torch tqdm pandas matplotlib seaborn scikit-learn
print("✅ ライブラリのインストールが完了しました。")

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm

## 📍 ステップ1: 単語の「順序」のベクトル化 (Positional Encoding)

第1課題(ステップ1)では、「単語そのものの意味を表す埋め込みベクトル」（`word_embeddings`。以下「**単語ベクトル**」）を学びました。これは文脈によらず固定的な、辞書的な意味を持ちます。

しかし、AIが文を理解するには「単語の意味」だけでは不十分です。「単語の**順序**」が不可欠です。
* `man bites dog` (人が犬を噛む)
* `dog bites man` (人が犬を噛む)
使われている単語は同じですが、順序が違うだけで意味が全く異なります。

`Transformer` は、この「順序」の情報を `Attention` 層に伝えるため、各単語の「単語ベクトル」に、その単語の「**位置ベクトル（Positional Encoding; PE）**」を**足し算**します。

${\bf InputVector}_i = \text{WordEmbedding}(\boldsymbol{x}_i) + \text{PositionalEncoding}(i)$

（`Attention` の本題（QKV）に入る前に、この「入力の下ごしらえ」であるPEについて解説します）

### なぜ「足し算」か？（Concatではなく）

「単語ベクトル」と「位置ベクトル」を「連結（Concat）」するのではなく、「足し算（Sum）」するのには、主に2つの理由があります。

1.  **計算効率（次元数の維持）:**
    * `Concat`（連結）の場合:
        `単語(768次元)` + `位置(768次元)` を連結すると、入力ベクトルが **1536次元** になってしまいます。後続の全結合層（$W_q, W_k, W_v$）のパラメータ数（計算コスト）が激増します。
    * `Sum`（足し算）の場合:
        `単語(768次元)` + `位置(768次元)` $\rightarrow$ **768次元** のままです。次元数が変わらないため、後続の層は元のサイズのままで済み、効率的です。

2.  **モデルの柔軟性（重ね合わせ）:**
    「意味」と「位置」が重ね合わさったベクトルを入力として受け取ることで、後続の全結合層（$W_q$ 等）が、学習を通じて「どの程度意味を重視し、どの程度位置を重視するか」を柔軟に統合して学習できます。

### `sin/cos`（三角関数）による計算

（BERTやGPT-2は「位置ベクトル」を学習しますが）オリジナルのTransformerでは、`sin` と `cos` の周期的な関数を用いてPEベクトルを決定論的に計算します。

* ベクトルの**偶数**次元（$2i$）：$PE_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{\text{model}}})$
* ベクトルの**奇数**次元（$2i+1$）：$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{\text{model}}})$

$pos$（位置）が変われば `sin/cos` の値が変わり、$i$（ベクトルの次元）が変われば関数の「周期」が変わります。これにより、各位置に固有のベクトルが割り当てられます。

以下のコードで、この `sin/cos` 関数のグラフと、実際のベクトル値を確認します。

In [97]:
#@title 模式図を表示
from IPython.display import display, HTML

svg_code_string = """
<svg width="1200px" viewBox="0 0 800 500" xmlns="http://www.w3.org/2000/svg">
  <!-- 定義 -->
  <defs>
    <!-- グラデーション: 単語ベクトル (man用) -->
    <linearGradient id="wordGradMan" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5BA3F5;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 単語ベクトル (bites用) -->
    <linearGradient id="wordGradBites" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#9B59B6;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#B87FCB;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 単語ベクトル (dog用) -->
    <linearGradient id="wordGradDog" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#52BE80;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 位置ベクトル -->
    <linearGradient id="posGrad1" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#F5A623;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F8B849;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="posGrad2" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#E85D75;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F07A8E;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="posGrad3" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#7B68EE;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#9B88FF;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 結果ベクトル (man + PE(0)) -->
    <linearGradient id="resultGradManPos0" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="50%" style="stop-color:#8DAFD7;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F5A623;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 結果ベクトル (bites + PE(1)) -->
    <linearGradient id="resultGradBitesPos1" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#9B59B6;stop-opacity:1" />
      <stop offset="50%" style="stop-color:#C07896;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#E85D75;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 結果ベクトル (dog + PE(2)) -->
    <linearGradient id="resultGradDogPos2" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="50%" style="stop-color:#5BAB97;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#7B68EE;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 結果ベクトル (man + PE(2)) -->
    <linearGradient id="resultGradManPos2" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="50%" style="stop-color:#618CB9;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#7B68EE;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: 結果ベクトル (dog + PE(0)) -->
    <linearGradient id="resultGradDogPos0" x1="0%" y1="0%" x2="100%" y2="0%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="50%" style="stop-color:#8EAA42;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F5A623;stop-opacity:1" />
    </linearGradient>
  </defs>

  <!-- タイトル -->
  <text x="400" y="25" font-size="18" font-weight="bold" text-anchor="middle" fill="#333">
    Positional Encoding: 位置情報の付加
  </text>

  <!-- 例文1: man bites dog -->
  <g id="sentence1">
    <!-- 文のラベル -->
    <text x="50" y="70" font-size="14" font-weight="bold" fill="#333">"man bites dog"</text>

    <!-- 縦ベクトルのカッコ (word vec) -->
    <path d="M -5,100 L -5,95 L -10,95 M -10,95 L -10,240 M -10,240 L -5,240 L -5,235"
          stroke="#333" stroke-width="2" fill="none" transform="translate(50, 0)"/>

    <!-- 縦ベクトルのカッコ (input vec) -->
    <path d="M 305,100 L 305,95 L 310,95 M 310,95 L 310,240 M 310,240 L 305,240 L 305,235"
          stroke="#333" stroke-width="2" fill="none" transform="translate(50, 0)"/>

    <!-- man (位置0) -->
    <g id="man_pos0" transform="translate(50, 90)">
      <text x="0" y="0" font-size="12" fill="#666">man (pos=0)</text>

      <!-- 単語ベクトル -->
      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradMan)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <!-- + 記号 -->
      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <!-- 位置ベクトル(0) -->
      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad1)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(0)</text>

      <!-- = 記号 -->
      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <!-- 結果ベクトル -->
      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradManPos0)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>

    <!-- bites (位置1) -->
    <g id="bites_pos1" transform="translate(50, 145)">
      <text x="0" y="0" font-size="12" fill="#666">bites (pos=1)</text>

      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradBites)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad2)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(1)</text>

      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradBitesPos1)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>

    <!-- dog (位置2) -->
    <g id="dog_pos2" transform="translate(50, 200)">
      <text x="0" y="0" font-size="12" fill="#666">dog (pos=2)</text>

      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradDog)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad3)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(2)</text>

      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradDogPos2)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>
  </g>

  <!-- 区切り線 -->
  <line x1="30" y1="260" x2="770" y2="260" stroke="#CCC" stroke-width="2" stroke-dasharray="5,5"/>

  <!-- 例文2: dog bites man -->
  <g id="sentence2">
    <!-- 文のラベル -->
    <text x="450" y="70" font-size="14" font-weight="bold" fill="#333">"dog bites man"</text>

    <!-- 縦ベクトルのカッコ (word vec) -->
    <path d="M -5,100 L -5,95 L -10,95 M -10,95 L -10,240 M -10,240 L -5,240 L -5,235"
          stroke="#333" stroke-width="2" fill="none" transform="translate(450, 0)"/>

    <!-- 縦ベクトルのカッコ (input vec) -->
    <path d="M 305,100 L 305,95 L 310,95 M 310,95 L 310,240 M 310,240 L 305,240 L 305,235"
          stroke="#333" stroke-width="2" fill="none" transform="translate(450, 0)"/>

    <!-- dog (位置0) - 単語は同じでも位置が異なる -->
    <g id="dog_pos0" transform="translate(450, 90)">
      <text x="0" y="0" font-size="12" fill="#666">dog (pos=0)</text>

      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradDog)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <!-- 位置0なので位置ベクトルはmanと同じ色 -->
      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad1)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(0)</text>

      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradDogPos0)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>

    <!-- bites (位置1) -->
    <g id="bites_pos1_2" transform="translate(450, 145)">
      <text x="0" y="0" font-size="12" fill="#666">bites (pos=1)</text>

      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradBites)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad2)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(1)</text>

      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradBitesPos1)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>

    <!-- man (位置2) - 単語は同じでも位置が異なる -->
    <g id="man_pos2" transform="translate(450, 200)">
      <text x="0" y="0" font-size="12" fill="#666">man (pos=2)</text>

      <rect x="0" y="10" width="80" height="25" fill="url(#wordGradMan)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="40" y="27" font-size="10" text-anchor="middle" fill="white">word vec</text>

      <text x="90" y="27" font-size="16" font-weight="bold" fill="#333">+</text>

      <!-- 位置2なので位置ベクトルはdogと同じ色 -->
      <rect x="110" y="10" width="80" height="25" fill="url(#posGrad3)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="150" y="27" font-size="10" text-anchor="middle" fill="white">PE(2)</text>

      <text x="200" y="27" font-size="16" font-weight="bold" fill="#333">=</text>

      <rect x="220" y="10" width="80" height="25" fill="url(#resultGradManPos2)" stroke="#333" stroke-width="1" rx="3"/>
      <text x="260" y="27" font-size="10" text-anchor="middle" fill="white">input vec</text>
    </g>
  </g>

  <!-- 説明テキスト -->
  <g id="explanation">
    <rect x="30" y="280" width="740" height="200" fill="#F8F9FA" stroke="#DDD" stroke-width="1" rx="5"/>

    <text x="400" y="305" font-size="14" font-weight="bold" text-anchor="middle" fill="#333">
      重要なポイント
    </text>

    <g id="point1" transform="translate(50, 325)">
      <circle cx="0" cy="0" r="4" fill="#4A90E2"/>
      <text x="15" y="5" font-size="12" fill="#333">
        <tspan font-weight="bold">単語ベクトル</tspan>は位置に関係なく同じ（"man"はどこでも同じベクトル）
      </text>
    </g>

    <g id="point2" transform="translate(50, 355)">
      <circle cx="0" cy="0" r="4" fill="#F5A623"/>
      <text x="15" y="5" font-size="12" fill="#333">
        <tspan font-weight="bold">位置ベクトル</tspan>は位置ごとに異なる（PE(0), PE(1), PE(2)はそれぞれ異なる）
      </text>
    </g>

    <g id="point3" transform="translate(50, 385)">
      <circle cx="0" cy="0" r="4" fill="#8DAFD7"/>
      <text x="15" y="5" font-size="12" fill="#333">
        両者を<tspan font-weight="bold">足し合わせる</tspan>ことで、「単語の意味」と「文中の位置」を同時に表現
      </text>
    </g>

    <g id="point4" transform="translate(50, 415)">
      <text x="0" y="5" font-size="12" fill="#333">
        → 同じ単語でも<tspan font-weight="bold">位置が違えば異なる入力ベクトル</tspan>になる
      </text>
    </g>

    <g id="point5" transform="translate(50, 445)">
      <text x="0" y="5" font-size="12" fill="#666">
        例: 左の"man"(pos=0)と右の"man"(pos=2)は、単語は同じでも異なる入力ベクトルになる
      </text>
    </g>
  </g>
</svg>
"""

# HTML() を使って表示します
display(HTML(svg_code_string))

In [ ]:
#@title ステップ1: Positional Encoding (PE) の計算とベクトル値の表示
#@markdown ---
#@markdown ### フォーム
#@markdown PEベクトルを計算したい「入力文」と、着目する「単語の位置（何番目か）」を指定してください。
sample_text_pe = "This is an orange" #@param {type:"string"}
target_position = 0 #@param {type:"slider", min:0, max:50, step:1}
#@markdown 表示する次元数（最初と最後）
display_dims = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown ---

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm

# --- GPT-2のロード（単語ベクトル(wte)取得のため） ---
if 'tokenizer_gpt' not in globals():
    tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
    model_gpt = GPT2Model.from_pretrained("gpt2").to(device)
    print("GPT-2 モデルとトークナイザをロードしました。")
else:
    print("GPT-2 モデルはロード済みです。")

d_model = model_gpt.config.n_embd # GPT-2の次元数 (768)

# --- 1. sin/cos を使ったPEベクトルの計算関数 ---
def get_positional_encoding(position, d_model):
    """
    sin/cos を使って指定された位置(position)のPEベクトル(d_model次元)を計算する
    """
    # PE計算用の角度の分母 (10000^(2i/d_model))
    # div_term の形状は (d_model / 2) = 384
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))

    pe = torch.zeros(d_model)

    # 偶数インデックス (2i) には sin
    pe[0::2] = torch.sin(position * div_term)
    # 奇数インデックス (2i + 1) には cos
    pe[1::2] = torch.cos(position * div_term)

    return pe.to(device)

# --- 2. 入力文をトークナイズ ---
inputs = tokenizer_gpt(sample_text_pe, return_tensors="pt").to(device)
token_ids = inputs['input_ids'][0] # (SeqLen)
seq_len = token_ids.shape[0]

# フォームで指定された位置が文の長さを超えていないかチェック
if target_position >= seq_len:
    print(f"エラー: 'target_position' ({target_position}) が文の長さ ({seq_len}) を超えています。")
    print(f"最大位置は {seq_len - 1} です。0に変更します。")
    target_position = 0

tokens_display = [tokenizer_gpt.decode([t]) for t in token_ids]
target_token_id = token_ids[target_position]
target_token_str = tokenizer_gpt.decode([target_token_id])

print(f"入力文: {sample_text_pe}")
print(f"トークン: {tokens_display} (文の長さ: {seq_len})")
print(f"着目する位置: {target_position} (単語: '{target_token_str}')")

# --- 3. PE関数の可視化 (グラフ) ---
plt.figure(figsize=(12, 4))
sns.set_style("whitegrid")

# 文の長さ (seq_len) までのPEベクトルを計算してプロット
# (最大5件までプロット)
plot_len = min(seq_len, 5)
for pos in range(plot_len):
    pe_vec = get_positional_encoding(pos, d_model).cpu().numpy()
    plt.plot(pe_vec[:100], label=f"PE (pos={pos}, word='{tokens_display[pos]}') [dim 0-99]",
             linestyle=('--' if pos != target_position else '-'),
             linewidth=(1 if pos != target_position else 2.5))

plt.title(f"Positional Encoding (sin/cos) values for this sentence (Plotting pos 0-{plot_len-1})")
plt.xlabel("Vector Dimension (i)")
plt.ylabel("Value (sin or cos)")
plt.legend()
plt.show()
print(f"➡️ 文の長さ ({seq_len}) に応じて、各位置 (0〜{seq_len-1}) に異なるベクトルが割り当てられます。")
print(f"   (グラフは着目位置 {target_position} を実線で、他を破線で表示しています)")


# --- 4. 単語ベクトルとPEベクトルの取得と表示 ---
print("\n" + "="*30)
print(f"'{target_token_str}' (位置 {target_position}) のベクトル計算")
print("="*30)

# (a) 単語ベクトル (WTE)
with torch.no_grad():
    word_embedding = model_gpt.wte(target_token_id.unsqueeze(0)).squeeze(0) # (768)
print(f"'{target_token_str}' (ID: {target_token_id.item()}) の単語ベクトルを取得。")

# (b) 位置ベクトル (PE)
positional_encoding = get_positional_encoding(target_position, d_model)
print(f"位置 {target_position} のPEベクトルを計算。")

# (c) 足し算 (Input Vector)
input_vector = word_embedding + positional_encoding
print("2つのベクトルを足し算しました。")

# --- 5. ベクトル値のテキスト表示 ---
def format_vec_ends(vec, n_dims):
    # ベクトルの最初と最後のn_dims次元を表示
    if vec.shape[0] <= n_dims * 2:
        return f"{vec}"
    start = ", ".join([f"{x:.4f}" for x in vec[:n_dims]])
    end = ", ".join([f"{x:.4f}" for x in vec[-n_dims:]])
    return f"[{start}, ..., {end}]"

print("\n--- ベクトル値の比較 (着目位置: {target_position}) ---")
print(f"(A) Word ('{target_token_str}'): {format_vec_ends(word_embedding.cpu(), display_dims)}")
print(f"(B) PE (pos={target_position}) : {format_vec_ends(positional_encoding.cpu(), display_dims)}")
print(f"(A+B) Result Vector: {format_vec_ends(input_vector.cpu(), display_dims)}")

print("\n➡️ 「単語ベクトル」(A) に「PEベクトル」(B) が足され、位置情報が反映された「入力ベクトル」(A+B) が生成されました。")
print("   この (A+B) が、次のステップ (QKV層) への入力となります。")

> 👩‍🎓 **課題 (ステップ1 考察)**
>
> 1.  上のセルのフォーム機能を使って、`target_word` と `position` を変えてみましょう。
> 2.  `position` を変えると、「PE (pos=...)」の点（緑のX）と「Result (Word + PE)」の点（赤のP）の位置はどのように変わるか観察してください。
> 3.  `target_word` を変えると、「Word ('...')」の点（青のo）と「Result (Word + PE)」の点（赤のP）の位置はどのように変わるか観察してください。
> 4.  （考察）`Attention` 層が受け取るのは、この「Result (Word + PE)」ベクトルです。なぜこのベクトルが `Concat`（連結）ではなく `Sum`（足し算）で計算されるのか、解説文を読んであなたの言葉で説明してください。（自由記述）

## 💡 ステップ2: 3つの役割への変換 (Wq, Wk, Wv)

ステップ1で、「単語ベクトル」と「位置ベクトル」を足し合わせた「**入力ベクトル**」（$x$）が完成しました。

第1課題では、この入力ベクトルを `Attention` が「よしなに処理して」文脈ベクトルを作ると説明しました。しかし、`Attention` は「入力ベクトル」を**そのまま使いません**。

`Attention` の核心は、入力された「入力ベクトル」（$x$）を、3つの異なる**全結合層（Linear Layer）**（数学的には行列 $W_q, W_k, W_v$ との積）に通すことにあります。

この計算により、3つの**全く異なる**ベクトル（Q, K, V）が生成されます。

* $q = \text{Linear}_q(x) = {\bf x}  W_q$
* $k = \text{Linear}_k(x) = x  W_k$
* $v = \text{Linear}_v(x) = x  W_v$

### なぜ3つに分けるのか？ (QKVすべてが関わる理由)

`Attention` の文脈ベクトルは、単なる『生の埋め込みベクトル』の重み付け平均ではありません。もし「生の入力ベクトル $x$」をそのまま重み付け平均すると、「問い合わせる役割」も「検索される役割」も「渡す情報」も、すべて同じベクトル $x$ が担うことになり、柔軟性がありません。

`Transformer` は、これら3つの役割を「学習可能な」全結合層（$W_q, W_k, W_v$）として分離しています。

* **Q (Query) $\leftarrow W_q$**

    「次単語予測」のタスクを解くために、「私は今、文法的な関係を**問い合わせたい**」という役割に $x$ を変換します。
* **K (Key) $\leftarrow W_k$**

     「『問い合わせ』に**応答する**（検索キー）」という役割に $x$ を変換します。（例: `Query(動詞)` に応答する `Key(名詞)`）

* **V (Value) $\leftarrow W_v$**

    「文脈として**渡すべき情報**」に $x$ を変換します。（例: `river` という単語から「水」「流れ」という情報だけを抽出する）

`Attention` の計算（ステップ3以降）は、この「タスク用に変換された」Q, K, V ベクトルを使って行われます。

In [ ]:
#@title ステップ2: QKV変換用 全結合層の確認
#@markdown ---
#@markdown ### フォーム
#@markdown QKVへの変換をシミュレートする入力文を指定してください。
sample_text = "Apple Store" #@param {type:"string"}
#@markdown ---

# --- GPT-2モデルのロード (ロード済みか確認) ---
if 'model_gpt' not in globals():
    tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
    model_gpt = GPT2Model.from_pretrained("gpt2").to(device)
    print("GPT-2 モデルとトークナイザをロードしました。")
else:
    print("GPT-2 モデルはロード済みです。")

d_model = model_gpt.config.n_embd # 768

# --- 1. Transformerの第1ブロック(h.0)にあるQKV全結合層を取得 ---
# (GPT-2では Q, K, V が c_attn という1つの大きな全結合層に統合されています)
qkv_layer = model_gpt.h[0].attn.c_attn

print(f"--- GPT-2 Layer 0: QKV全結合層 (c_attn) ---")
print(f"層のタイプ: {type(qkv_layer)}")

# 'Conv1D' object には 'in_features' / 'out_features' がありません。
# 'Conv1D' の .weight は (in_features, out_features) の形状で保存されています。
in_dim = qkv_layer.weight.shape[0]
out_dim = qkv_layer.weight.shape[1]

print(f"入力次元 (d_model): {in_dim}")
print(f"出力次元 (d_model * 3): {out_dim}")
print(f"({d_model}次元 * 3 = {d_model*3})")

# --- 2. ステップ1の処理 (単語ベクトル + 位置ベクトル) ---
# (GPT-2は学習されたPE(wpe)を使うため、ステップ1のsin/cos関数ではなくモデルのwpe層を使います)
inputs = tokenizer_gpt(sample_text, return_tensors="pt").to(device)
token_ids = inputs['input_ids']
positions = torch.arange(0, token_ids.shape[1]).to(device)

with torch.no_grad():
    word_embeddings = model_gpt.wte(token_ids) # (Batch, SeqLen, 768)
    position_embeddings = model_gpt.wpe(positions) # (SeqLen, 768)

    # ブロードキャストを利用して足し算
    input_vectors = word_embeddings + position_embeddings # (Batch, SeqLen, 768)

print(f"\n--- 入力 ---")
print(f"入力文: {sample_text}")
print(f"入力ベクトルの形状: {input_vectors.shape}")

# --- 3. QKV全結合層に入力 ---
with torch.no_grad():
    qkv_vectors = qkv_layer(input_vectors) # (Batch, SeqLen, 2304)

print(f"\n--- 出力 (QKV変換後) ---")
print(f"QKVベクトルの形状: {qkv_vectors.shape}")

# --- 4. Q, K, V への分割 ---
# (実際にはMulti-Headのためにさらに分割されますが、ここでは概念的に3つに分けます)
q_vec, k_vec, v_vec = qkv_vectors.split(d_model, dim=2)

print(f"\n--- 分割後 ---")
print(f"Query (q) ベクトルの形状: {q_vec.shape}")
print(f"Key (k) ベクトルの形状: {k_vec.shape}")
print(f"Value (v) ベクトルの形状: {v_vec.shape}")

print("\n➡️ 「単語+位置ベクトル」(768次元) が、Wq, Wk, Wv (全結合層) を通り、")
print("   「タスク指向の役割」(Q, K, V) を持つ3つの異なるベクトル（各768次元）に変換されました。")

## ✍️ ステップ3: 重みの計算 (Q・K 内積と Vの加重平均)

ステップ2で、タスク指向の $\boldsymbol{Q}$ (Query), $\boldsymbol{K}$ (Key), $\boldsymbol{V}$ (Value) ベクトルが準備できました。
`Attention` は、これらを使って「文脈ベクトル」を計算します。

### Attentionの計算プロセス（文章による説明）

`Attention` の計算プロセスは、各単語が「文脈ベクトル」を生成するための一連のステップです。このステップは、大きく「**重み（注目度）を決定する**」段階と、「**情報を集約する**」段階に分けることができます。

最初の「重み決定」段階では、ある単語（例: `Store`）が、文中の他のすべての単語（自分自身を含む）に対して、どれだけ注目すべきかを計算します。このために、`Store` は自分の「**Query（問い合わせ）**」ベクトル（$\boldsymbol{q}_{\text{Store}}$）を使います。この $\boldsymbol{q}_{\text{Store}}$ を、文中の全単語（`I`, `went`, `Apple`...）が持つ「**Key（名札・検索キー）**」ベクトル（$\boldsymbol{k}_I, \boldsymbol{k}_{went}, \boldsymbol{k}_{\text{Apple}}...$）と一つずつ照合します。この照合の結果、「どのKeyが自分のQueryと最も関連性が高いか」を示すスコアが算出されます。

次の「情報集約」段階では、このスコアを「重み」（合計1.0の確率）に変換します。この重みは、「どの単語から情報をどれだけの割合で取り込むか」という比率の役割を果たします。`Attention` はこの重みを用いて、各単語が持つ「**Value（渡すべき情報）**」ベクトル（$\boldsymbol{v}_I, \boldsymbol{v}_{went}, \boldsymbol{v}_{\text{Apple}}...$）を**加重平均**（重み付けして足し合わせる）します。`Store` の最終的な「文脈ベクトル」は、例えば「`Apple` の $\boldsymbol{V}$ ベクトルを 88%ぶん」、「`I` の $\boldsymbol{V}$ ベクトルを 1%ぶん」... というように、注目度（重み）に応じて情報（$\boldsymbol{V}$ ベクトル）を混ぜ合わせたものになります。

---

### (a) 内積と「近さ（Attentionスコア）」の関係

`Attention` の「重み」は、ステップ2で得られた「**$\boldsymbol{Q}$ベクトル**」と「**$\boldsymbol{K}$ベクトル**」の「近さ」によって決まります。

* `I went to the Apple Store` という文で、`Store` の文脈ベクトルを作りたいとします。
* `Store` の **$\boldsymbol{Q}$ベクトル** ($\boldsymbol{q}_{\text{Store}}$) を取り出します。
* この $\boldsymbol{q}_{\text{Store}}$ と、文中のすべての単語の **$\boldsymbol{K}$ベクトル**（$\boldsymbol{k}_I, \boldsymbol{k}_{went}, \dots, \boldsymbol{k}_{\text{Store}}$）との「**内積（Dot Product）**」を計算します。

> **内積と「近さ」**
>
> 2つのベクトル $\boldsymbol{q}$ と $\boldsymbol{k}$ の内積 $\boldsymbol{q} \cdot \boldsymbol{k}$ は、2つのベクトルの「方向」が似ている（$\cos\theta$ が 1 に近い）ほど値が大きくなります。（$\theta$ は2ベクトル間の角度）
>
> $W_q$ と $W_k$ は、「`Store` の $\boldsymbol{q}$」と「`Apple` の $\boldsymbol{k}$」が**似た方向を向くように**（＝内積が大きくなるように）学習されます。
>
> これが、第1課題の「辞書的な近さ（`grape` vs `orange`）」とは異なる、「**タスク指向の（文脈的な）近さ**」の正体です。

### (b) 文脈ベクトルの計算プロセス（数式による説明）

$\text{Attention}(\boldsymbol{Q}, \boldsymbol{K}, \boldsymbol{V}) = \text{Softmax}\left(\frac{\boldsymbol{Q}\boldsymbol{K}^T}{\sqrt{d_k}}\right) \boldsymbol{V}$

1.  **スコア計算 ($\boldsymbol{Q}\boldsymbol{K}^T$):**

`Store` の $\boldsymbol{q}$ と、全単語の $\boldsymbol{k}$ の**内積**を計算します。（例: `[0.1, 0.2, 0.9, 8.5, 4.0]`）

2.  **スケーリング ($\sqrt{d_k}$):**

内積の値が大きくなりすぎるのを防ぐため、ベクトルの次元数の平方根（$\sqrt{d_k}$）で割ります。（数式上の調整です）

3.  **重み化 (Softmax):**

スコアを `Softmax` 関数に通し、合計が 1.0 になる「重み（Attention Weight）」に変換します。（例: `[0.01, 0.01, 0.02, 0.88, 0.08]`）

4.  **加重平均 ($\dots \boldsymbol{V}$):**

この「重み」を使って、「$\boldsymbol{V}$ベクトル」（ $ \boldsymbol{v}_I, \boldsymbol{v}_\text{went}, \dots, \boldsymbol{v}_{\text{Store}} $）を**加重平均**（重み付けして足し合わせる）します。

$$\boldsymbol{\text{ContextVector}}(\text{Store}) = (0.01 \boldsymbol{v}_I) + \dots + (0.88 \boldsymbol{v}_{\text{Apple}}) + (0.08 \boldsymbol{v}_{\text{Store}})$$

この「$\boldsymbol{V}$ベクトルの加重平均」こそが、`Attention` 層の **最終的な出力（文脈ベクトル）** です。

---
### GPTの制約：未来は見ない（Causal Attention）

第1課題でも触れたように、`GPT` は「次単語予測」モデルです。
`Store` の「次」の単語を予測するために `Store` の文脈ベクトルを作るとき、`Store` よりも「未来」にある単語（例: `to` や `buy`）の情報を参照してはいけません（答えの先見になってしまうため）。

このため、`GPT`（デコーダー）の `Attention` は、ステップ3-(1) の「スコア計算」において、**自分より未来にある単語へのスコアを強制的に 0 に近い値（計算上は $-\infty$）にします**。
これを「**Causal Masking（マスク）**」と呼びます。

この結果、次のコードセルで表示される `Attention` の重み（注目度）のヒートマップは、対角線より**右上の領域がすべて 0.00 に近い値**（マスクされている）になります。

In [ ]:
#@title ステップ3: Attentionの重み (Q・Kの近さ) の可視化
#@markdown ---
#@markdown ### フォーム
#@markdown Attentionの重みを可視化したい入力文を指定してください。
sample_text_attn = "I went to the Apple Store to buy an iPhone" #@param {type:"string"}
#@markdown ---

# --- GPT-2モデルのロード (output_attentions=True) ---
# (ステップ2でロード済みだが、設定を変えて再ロード)
tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt_with_attentions = GPT2Model.from_pretrained("gpt2", output_attentions=True).to(device)
model_gpt_with_attentions.eval() # 評価モード
print("GPT-2 モデル (Attention出力付き) をロードしました。")

# --- 入力 ---
inputs = tokenizer_gpt(sample_text_attn, return_tensors="pt").to(device)
tokens = [tokenizer_gpt.decode([token_id]) for token_id in inputs['input_ids'][0]]

# --- モデル実行 (Attentionを取得) ---
with torch.no_grad():
    outputs = model_gpt_with_attentions(**inputs)
    # attentions は (Layer, Batch, Head, SeqLen, SeqLen) のタプル
    attentions_layer_0 = outputs.attentions[0] # 最初のレイヤー(Layer 0)を取得

print(f"\n入力トークン: {tokens}")
print(f"取得したAttentionの形状 (Layer 0): {attentions_layer_0.shape}")
print(" (Batch=1, Heads=12, QuerySeq=9, KeySeq=9)")

# --- 最初のヘッド(Head 0)のAttention重みを取得 ---
# (GPT-2はCausal Masking(未来を見ない)がかかっています)
attn_head_0 = attentions_layer_0[0, 0, :, :].cpu().numpy() # (QuerySeq, KeySeq)

# --- 可視化 (ヒートマップ) ---
plt.figure(figsize=(10, 8))
sns.heatmap(attn_head_0,
            annot=True,
            fmt=".2f",
            cmap="viridis",
            xticklabels=tokens,
            yticklabels=tokens)
plt.title("Attention Weights (Q x K Inner Product) - Layer 0, Head 0")
plt.xlabel("Key")
plt.ylabel("Query")
plt.show()

print("\n➡️ ヒートマップの見方:")
print("・各行 (Y軸) が「Query（問い合わせる単語）」です。")
print("・各列 (X軸) が「Key（応答する単語）」です。")
print(f"・例えば、行 '{tokens[5]}' (Store) を見ると、列 '{tokens[4]}' (Apple) の値が他と比べて高くなっています。")
print("・これは「StoreのQ」と「AppleのK」の内積（近さ）が、タスクを解く上で関連性が高いと学習されたことを示します。")
print("・右上がりの対角線より上が 0.00 に近いのは、GPTが「未来」を見ないようにマスク(Causal Mask)をかけているためです（詳細はステップ5で解説）。")

> 👩‍🎓 **課題 (ステップ3 考察)**
>
> 1.  上のセルのフォーム機能を使って、`sample_text_attn` をあなたが考えた別の文（例: `The quick brown fox jumps over the lazy dog`）に変更し、ヒートマップがどう変わるか観察してください。
> 2.  ヒートマップを見て、ある単語（Query）が、どの単語（Key）に強く注目しているか（＝Q・Kの内積が大きいか）を2～3例、挙げてください。
> 3.  （考察）`Attention` の「重み」は、第1課題の「辞書的な近さ（`grape` と `orange`）」と、何が違うのでしょうか？ ステップ2, 3の解説を読んであなたの言葉で説明してください。（自由記述）

## 🚀 ステップ4: Multi-Head Attention (並列化の意義)

ステップ3では、`Attention` がQKVの1セット（1つのヘッド）で計算される様子を見ました。
しかし、1セットのQKV（$W_q, W_k, W_v$）だけでは、1種類の関係性（例: `Apple` $\rightarrow$ `Store` という固有名詞の関係）しか学習できません。

`Multi-Head Attention` は、このQKVの計算セットを並列に（例: `GPT-2` では12個）実行します。

### 次元分割 (d_model / n_head)

ご指摘の通り、`Multi-Head Attention` は、入力ベクトルの次元（$d_{\text{model}}$, 例: 768）を、ヘッドの数（$n_{\text{head}}$, 例: 12）で**分割**し、各ヘッドに「分担」させます。

* 各ヘッドが扱う次元 $d_{\text{head}} = d_{\text{model}} / n_{\text{head}} = 768 / 12 = 64$ 次元。

ステップ2で見た `c_attn` 層（入力768 $\rightarrow$ 出力2304）は、内部的には、各ヘッド（12個）がそれぞれ「入力768 $\rightarrow$ 出力64*3 (QKV)」の計算を行う行列を保持していると解釈できます。

各「ヘッド」（Head 0〜11）は、**異なる** $W_q, W_k, W_v$ の行列（入力768 $\rightarrow$ 出力64）を持ち、**異なる種類の関係性**（異なる注目パターン）を、それぞれ64次元の空間で学習できます。

### 集約方法 (Concat + 全結合層)

1.  **並列計算:** 12個のヘッドが並列でAttention計算を行い、それぞれ**64次元**の出力ベクトル（$Z_0, Z_1, \dots, Z_{11}$）を生成します。
2.  **Concat（連結）:** これら12個の「64次元」ベクトルが `concat`（連結）され、一つの大きなベクトル（$Z_{\text{concat}}$）になります。
    * $Z_{\text{concat}}$ の次元は $64 \times 12 = 768$ 次元（元の $d_{\text{model}$ に戻る）。
3.  **全結合層 ($W_o$):** この連結されたベクトル（768次元）を、もう一つの全結合層（$W_o$, 768 $\rightarrow$ 768）に通すことで、並列で得られたすべての知見を「統合」した、最終的な出力ベクトル（$Z_{\text{output}}$）に戻します。

次のコードで、異なるヘッド（Head）が、同じ入力文に対してどのように異なる注目パターンを学習しているか可視化します。

In [98]:
#@title 模式図を表示
from IPython.display import display, HTML

svg_code_string = """<svg width="1200px" viewBox="0 0 1200 700" xmlns="http://www.w3.org/2000/svg">
  <!-- 定義 -->
  <defs>
    <!-- グラデーション: 入力ベクトル -->
    <linearGradient id="inputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5BA3F5;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Query (Head 1) -->
    <linearGradient id="queryGrad1" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#E74C3C;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#EC7063;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Key (Head 1) -->
    <linearGradient id="keyGrad1" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#F39C12;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#F5B041;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Value (Head 1) -->
    <linearGradient id="valueGrad1" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#9B59B6;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#BB8FCE;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Query (Head 2) -->
    <linearGradient id="queryGrad2" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#C0392B;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#D98880;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Key (Head 2) -->
    <linearGradient id="keyGrad2" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#D68910;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#E59866;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Value (Head 2) -->
    <linearGradient id="valueGrad2" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#7D3C98;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#A569BD;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Attention Weights -->
    <linearGradient id="attnGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#16A085;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#48C9B0;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: Output -->
    <linearGradient id="outputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#52BE80;stop-opacity:1" />
    </linearGradient>

    <!-- グラデーション: FFN -->
    <linearGradient id="ffnGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#2E86C1;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5DADE2;stop-opacity:1" />
    </linearGradient>

    <!-- 矢印マーカー -->
    <marker id="arrowhead" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#555" />
    </marker>
  </defs>

  <!-- タイトル -->
  <text x="600" y="25" font-size="18" font-weight="bold" text-anchor="middle" fill="#333">
    Multi-Head Self-Attention (2 Heads)
  </text>

  <!-- ステップラベル -->
  <g id="step_labels">
    <text x="50" y="60" font-size="11" font-weight="bold" fill="#666">Step 1:</text>
    <text x="50" y="75" font-size="10" fill="#666">Embed + PE</text>

    <text x="390" y="60" font-size="13" font-weight="bold" fill="#666">Step 2: Multi-Head QKV</text>

    <text x="650" y="60" font-size="13" font-weight="bold" fill="#666">Step 3: Concat</text>

    <text x="820" y="60" font-size="13" font-weight="bold" fill="#666">Step 4: FFN</text>
  </g>

  <!-- テキスト入力 -->
  <g id="text_input" transform="translate(30, 200)">
    <rect x="0" y="0" width="60" height="200" fill="#F8F9FA" stroke="#CCC" stroke-width="1" rx="3"/>
    <text x="30" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="#333">Text</text>

    <text x="30" y="70" font-size="11" text-anchor="middle" fill="#555">"man</text>
    <text x="30" y="120" font-size="11" text-anchor="middle" fill="#555">bites</text>
    <text x="30" y="170" font-size="11" text-anchor="middle" fill="#555">dog"</text>
  </g>

  <!-- 矢印: Text → Embedding -->
  <path d="M 95 300 L 115 300" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <!-- 埋め込みベクトル -->
  <g id="embedding" transform="translate(120, 200)">
    <rect x="0" y="0" width="70" height="200" fill="#8E44AD" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Word</text>
    <text x="35" y="35" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Embed</text>

    <!-- 3つの単語を表す横線 -->
    <line x1="10" y1="70" x2="60" y2="70" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="120" x2="60" y2="120" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="170" x2="60" y2="170" stroke="white" stroke-width="1.5" opacity="0.6"/>
  </g>

  <!-- Positional Encoding (上から、Word Embedの真上) -->
  <g id="positional_encoding" transform="translate(120, 110)">
    <rect x="0" y="0" width="70" height="60" fill="#E67E22" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Positional</text>
    <text x="35" y="35" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Encoding</text>
    <text x="35" y="52" font-size="9" text-anchor="middle" fill="white">(PE)</text>
  </g>

  <!-- 矢印: PE → Word Embed (足し算) -->
  <path d="M 155 175 L 155 195" stroke="#E67E22" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <!-- + 記号 -->
  <circle cx="155" cy="185" r="12" fill="white" stroke="#333" stroke-width="1.5"/>
  <text x="155" y="191" font-size="16" font-weight="bold" text-anchor="middle" fill="#333">+</text>

  <!-- 矢印: Embedding → Input Vec -->
  <path d="M 195 300 L 215 300" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <!-- 入力ベクトル (Embedding + PE の結果) -->
  <g id="input_vec" transform="translate(220, 200)">
    <rect x="0" y="0" width="70" height="200" fill="url(#inputGrad)" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Input</text>
    <text x="35" y="35" font-size="10" text-anchor="middle" fill="white">Vec</text>

    <!-- 上半分を示す線 -->
    <line x1="5" y1="100" x2="65" y2="100" stroke="white" stroke-width="1" stroke-dasharray="2,2" opacity="0.8"/>
    <text x="35" y="60" font-size="9" text-anchor="middle" fill="white">upper</text>
    <text x="35" y="140" font-size="9" text-anchor="middle" fill="white">lower</text>
  </g>

  <!-- Transformer ブロックの囲み -->
  <rect x="305" y="100" width="600" height="450" fill="none" stroke="#2C3E50"
        stroke-width="3" rx="8" stroke-dasharray="8,4" opacity="0.8"/>
  <text x="677" y="95" font-size="13" font-weight="bold" text-anchor="middle" fill="#2C3E50">
    Transformer Block (Multi-Head Attention)
  </text>

  <!-- 矢印: Input Vec → 分岐点 -->
  <path d="M 295 300 L 330 300" stroke="#555" stroke-width="2" fill="none"/>

  <!-- 上下への分岐 -->
  <g id="head_split">
    <!-- 分岐点 -->
    <circle cx="335" cy="300" r="4" fill="#333"/>

    <!-- 分岐線 -->
    <path d="M 335 300 L 335 200" stroke="#555" stroke-width="2" fill="none"/>
    <path d="M 335 300 L 335 450" stroke="#555" stroke-width="2" fill="none"/>

    <!-- Head 1 (上) への矢印 -->
    <path d="M 335 200 L 370 200" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>
    <text x="340" y="185" font-size="10" font-weight="bold" fill="#333">Head 1</text>

    <!-- Head 2 (下) への矢印 -->
    <path d="M 335 450 L 370 450" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>
    <text x="340" y="465" font-size="10" font-weight="bold" fill="#333">Head 2</text>
  </g>

  <!-- ========== Head 1 (上) ========== -->
  <!-- Head 1: QKV 分岐 -->
  <g id="head1_qkv_split">
    <circle cx="390" cy="200" r="3" fill="#333"/>
    <path d="M 390 200 L 420 140" stroke="#E74C3C" stroke-width="1.5" fill="none"/>
    <path d="M 390 200 L 420 200" stroke="#F39C12" stroke-width="1.5" fill="none"/>
    <path d="M 390 200 L 420 260" stroke="#9B59B6" stroke-width="1.5" fill="none"/>
  </g>

  <!-- Head 1: Query -->
  <g id="query1" transform="translate(425, 120)">
    <rect x="0" y="0" width="50" height="40" fill="url(#queryGrad1)" stroke="#E74C3C" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Query</text>
  </g>

  <!-- Head 1: Key -->
  <g id="key1" transform="translate(425, 180)">
    <rect x="0" y="0" width="50" height="40" fill="url(#keyGrad1)" stroke="#F39C12" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Key</text>
  </g>

  <!-- Head 1: Value -->
  <g id="value1" transform="translate(425, 240)">
    <rect x="0" y="0" width="50" height="40" fill="url(#valueGrad1)" stroke="#9B59B6" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Value</text>
  </g>

  <!-- Head 1: Attention Weights -->
  <g id="attn_weights1" transform="translate(520, 145)">
    <rect x="0" y="0" width="65" height="50" fill="url(#attnGrad)" stroke="#16A085" stroke-width="1.5" rx="3"/>
    <text x="32.5" y="20" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Attention</text>
    <text x="32.5" y="33" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Weights</text>
  </g>

  <!-- Head 1: 矢印 Q,K → Attention Weights -->
  <path d="M 478 140 L 515 165" stroke="#555" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>
  <path d="M 478 200 L 515 180" stroke="#555" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 1: 重み付き和 -->
  <ellipse cx="552" cy="240" rx="30" ry="25" fill="#F0F0F0" stroke="#333" stroke-width="1.5"/>
  <text x="552" y="245" font-size="14" font-weight="bold" text-anchor="middle" fill="#333">×</text>

  <!-- Head 1: 矢印 Weights → 重み付き和 -->
  <path d="M 552 200 L 552 210" stroke="#16A085" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 1: 矢印 Value → 重み付き和 -->
  <path d="M 478 260 L 520 248" stroke="#9B59B6" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 1: Attention Output -->
  <g id="attn_output1" transform="translate(600, 220)">
    <rect x="0" y="0" width="50" height="50" fill="url(#outputGrad)" stroke="#27AE60" stroke-width="1.5" rx="3"/>
    <text x="25" y="22" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Output</text>
    <text x="25" y="34" font-size="9" text-anchor="middle" fill="white">1</text>
  </g>

  <!-- Head 1: 矢印 重み付き和 → Output -->
  <path d="M 585 245 L 595 245" stroke="#27AE60" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- ========== Head 2 (下) ========== -->
  <!-- Head 2: QKV 分岐 -->
  <g id="head2_qkv_split">
    <circle cx="390" cy="450" r="3" fill="#333"/>
    <path d="M 390 450 L 420 390" stroke="#C0392B" stroke-width="1.5" fill="none"/>
    <path d="M 390 450 L 420 450" stroke="#D68910" stroke-width="1.5" fill="none"/>
    <path d="M 390 450 L 420 510" stroke="#7D3C98" stroke-width="1.5" fill="none"/>
  </g>

  <!-- Head 2: Query -->
  <g id="query2" transform="translate(425, 370)">
    <rect x="0" y="0" width="50" height="40" fill="url(#queryGrad2)" stroke="#C0392B" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Query</text>
  </g>

  <!-- Head 2: Key -->
  <g id="key2" transform="translate(425, 430)">
    <rect x="0" y="0" width="50" height="40" fill="url(#keyGrad2)" stroke="#D68910" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Key</text>
  </g>

  <!-- Head 2: Value -->
  <g id="value2" transform="translate(425, 490)">
    <rect x="0" y="0" width="50" height="40" fill="url(#valueGrad2)" stroke="#7D3C98" stroke-width="1.5" rx="3"/>
    <text x="25" y="25" font-size="10" font-weight="bold" text-anchor="middle" fill="white">Value</text>
  </g>

  <!-- Head 2: Attention Weights -->
  <g id="attn_weights2" transform="translate(520, 395)">
    <rect x="0" y="0" width="65" height="50" fill="url(#attnGrad)" stroke="#16A085" stroke-width="1.5" rx="3"/>
    <text x="32.5" y="20" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Attention</text>
    <text x="32.5" y="33" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Weights</text>
  </g>

  <!-- Head 2: 矢印 Q,K → Attention Weights -->
  <path d="M 478 390 L 515 415" stroke="#555" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>
  <path d="M 478 450 L 515 430" stroke="#555" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 2: 重み付き和 -->
  <ellipse cx="552" cy="490" rx="30" ry="25" fill="#F0F0F0" stroke="#333" stroke-width="1.5"/>
  <text x="552" y="495" font-size="14" font-weight="bold" text-anchor="middle" fill="#333">×</text>

  <!-- Head 2: 矢印 Weights → 重み付き和 -->
  <path d="M 552 450 L 552 460" stroke="#16A085" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 2: 矢印 Value → 重み付き和 -->
  <path d="M 478 510 L 520 498" stroke="#7D3C98" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- Head 2: Attention Output -->
  <g id="attn_output2" transform="translate(600, 470)">
    <rect x="0" y="0" width="50" height="50" fill="url(#outputGrad)" stroke="#27AE60" stroke-width="1.5" rx="3"/>
    <text x="25" y="22" font-size="9" font-weight="bold" text-anchor="middle" fill="white">Output</text>
    <text x="25" y="34" font-size="9" text-anchor="middle" fill="white">2</text>
  </g>

  <!-- Head 2: 矢印 重み付き和 → Output -->
  <path d="M 585 495 L 595 495" stroke="#27AE60" stroke-width="1.5" fill="none" marker-end="url(#arrowhead)"/>

  <!-- ========== Concat ========== -->
  <!-- Output1とOutput2の結合 -->
  <g id="concat">
    <circle cx="680" cy="350" r="3" fill="#333"/>
    <path d="M 655 245 L 680 350" stroke="#27AE60" stroke-width="2" fill="none"/>
    <path d="M 655 495 L 680 350" stroke="#27AE60" stroke-width="2" fill="none"/>
  </g>

  <!-- Concat ボックス -->
  <g id="concat_box" transform="translate(685, 325)">
    <rect x="0" y="0" width="70" height="50" fill="#F8F9FA" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="25" font-size="11" font-weight="bold" text-anchor="middle" fill="#333">Concat</text>
    <text x="35" y="38" font-size="9" text-anchor="middle" fill="#555">(merge)</text>
  </g>

  <!-- 矢印: Concat → FFN -->
  <path d="M 760 350 L 790 350" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <!-- FFN (Feed Forward Network) -->
  <g id="ffn" transform="translate(795, 300)">
    <rect x="0" y="0" width="90" height="100" fill="url(#ffnGrad)" stroke="#2E86C1" stroke-width="2" rx="3"/>
    <text x="45" y="23" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Feed Forward</text>
    <text x="45" y="38" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Network</text>
    <text x="45" y="55" font-size="9" text-anchor="middle" fill="white">(2-layer MLP)</text>

    <!-- 3つの単語を表す横線 -->
    <line x1="12" y1="70" x2="78" y2="70" stroke="white" stroke-width="1" opacity="0.6"/>
    <line x1="12" y1="80" x2="78" y2="80" stroke="white" stroke-width="1" opacity="0.6"/>
    <line x1="12" y1="90" x2="78" y2="90" stroke="white" stroke-width="1" opacity="0.6"/>
  </g>

  <!-- 矢印: FFN → Block Output -->
  <path d="M 890 350 L 960 350" stroke="#27AE60" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <!-- ... (繰り返しの意) -->
  <text x="980" y="350" font-size="20" font-weight="bold" text-anchor="middle" fill="#666">...</text>

  <!-- LM Head -->
  <g id="lm_head" transform="translate(1010, 310)">
    <rect x="0" y="0" width="80" height="80" fill="#E74C3C" stroke="#333" stroke-width="2" rx="3"/>
    <text x="40" y="30" font-size="11" font-weight="bold" text-anchor="middle" fill="white">LM Head</text>
    <text x="40" y="48" font-size="9" text-anchor="middle" fill="white">(next word</text>
    <text x="40" y="61" font-size="9" text-anchor="middle" fill="white">prediction)</text>
  </g>

  <!-- 説明テキスト -->
  <g id="explanation">
    <rect x="30" y="620" width="1140" height="70" fill="#F8F9FA" stroke="#DDD" stroke-width="1" rx="5"/>

    <text x="600" y="645" font-size="13" font-weight="bold" text-anchor="middle" fill="#333">
      Multi-Head Attentionの処理の流れ
    </text>

    <g id="flow1" transform="translate(50, 665)">
      <circle cx="0" cy="0" r="3" fill="#4A90E2"/>
      <text x="12" y="4" font-size="11" fill="#333">
        Input Vec → 上半分がHead 1へ、下半分がHead 2へ分岐 → 各ヘッドでQuery, Key, Valueを生成してAttention計算
      </text>
    </g>
  </g>
</svg>
"""

# HTML() を使って表示します
display(HTML(svg_code_string))

In [ ]:
#@title ステップ4: Multi-Head Attention の可視化
#@markdown ---
#@markdown ### フォーム
#@markdown Attentionの重みを可視化したい入力文を指定してください。
sample_text_multi = "The quick brown fox jumps over the lazy dog" #@param {type:"string"}
#@markdown 表示したいヘッド番号 (0～11) を選んでください。
head_index = 0 #@param {type:"slider", min:0, max:11, step:1}
#@markdown ---

# --- GPT-2モデルのロード (ロード済みか確認) ---
if 'model_gpt_with_attentions' not in globals():
    tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
    model_gpt_with_attentions = GPT2Model.from_pretrained("gpt2", output_attentions=True).to(device)
    model_gpt_with_attentions.eval()
    print("GPT-2 モデル (Attention出力付き) をロードしました。")

# ▼▼▼【修正点】モデルの次元情報を取得 ▼▼▼
d_model = model_gpt_with_attentions.config.n_embd      # 768
n_head = model_gpt_with_attentions.config.n_head        # 12
d_head = d_model // n_head                              # 64 (768 / 12)

print(f"\n--- Multi-Head Attention の設定 ---")
print(f"モデル次元 (d_model): {d_model}")
print(f"ヘッド数 (n_head): {n_head}")
print(f"各ヘッドが担当する次元 (d_head = d_model / n_head): {d_head}")
print(f" (Head 0 は 0-{d_head-1}次元、Head 1 は {d_head}-{d_head*2-1}次元 ... の情報を主に処理します)")
# ▲▲▲ 修正完了 ▲▲▲

# --- 入力 ---
inputs = tokenizer_gpt(sample_text_multi, return_tensors="pt").to(device)
tokens = [tokenizer_gpt.decode([token_id]) for token_id in inputs['input_ids'][0]]

# --- モデル実行 (Attentionを取得) ---
with torch.no_grad():
    outputs = model_gpt_with_attentions(**inputs)
    attentions_layer_0 = outputs.attentions[0] # 最初のレイヤー(Layer 0)を取得

print(f"\n入力トークン: {tokens}")
print(f" (Y軸: Query, X軸: Key)") # 軸の対応を明記

# --- 指定されたヘッドのAttention重みを取得 ---
# (Batch=0, Head=head_index)
try:
    attn_head = attentions_layer_0[0, head_index, :, :].cpu().numpy() # (QuerySeq, KeySeq)

    # --- 可視化 (ヒートマップ) ---
    plt.figure(figsize=(10, 8))

    # xticklabels, yticklabels を False に設定
    sns.heatmap(attn_head,
                annot=True,
                fmt=".2f",
                cmap="viridis",
                xticklabels=False, # 警告を回避するため非表示
                yticklabels=False) # 警告を回避するため非表示

    # ▼▼▼【修正点】タイトルに次元情報を追加 ▼▼▼
    plt.title(f"Attention Weights (Layer 0, Head {head_index}) - Each Head processes {d_head} dims")
    plt.xlabel("Key (K) - Position (0 -> N)") # 英語ラベルに変更
    plt.ylabel("Query (Q) - Position (0 -> N)") # 英語ラベルに変更
    # ▲▲▲ 修正完了 ▲▲▲

    plt.show()

    print("\n➡️ フォームで 'head_index' を 0 から 11 まで変えて実行してみてください。")
    print("   ヘッドごとに注目しているパターン（ヒートマップの色の濃い場所）が異なることがわかります。")
    print("   (軸の単語ラベルは警告を避けるため非表示です。上記の「入力トークン」リストの順序に対応します)")

except IndexError as e:
    print(f"\nエラー: トークン化に失敗したか、指定したヘッドインデックスが不正です。 {e}")
except Exception as e:
    print(f"\nエラーが発生しました: {e}")

> 👩‍🎓 **課題 (ステップ4 考察)**
>
> 1.  上のセルのフォーム機能を使って、`sample_text_multi` を好きな文に変えてみましょう。
> 2.  `head_index` を 0, 1, 2... と変えて実行し、少なくとも2つの異なるヘッドが、それぞれどのような「注目パターン」を持っているか観察・記述してください。（例: 「Head 0 は常に2つ前の単語に注目している」「Head 5 は `jumps` の行で `fox` に強く注目している」など）
> 3.  （考察）なぜ `Transformer` は、1つの強力な `Attention` ではなく、複数の `Head` を並列で使うのでしょうか？ ステップ4の解説を読んであなたの言葉で説明してください。（自由記述）

## 🔩 ステップ5: FFN (Feed-Forward Network) の意義

`Transformer` の各ブロックは、`Attention` (MHA) だけで終わっていません。
`Attention` の出力（文脈が混合されたベクトル $Z_{\text{output}}$）は、次に **FFN (Feed-Forward Network)** というモジュールに入力されます。

### 全結合層 と FFN の違い

* **全結合層 (Linear Layer):** 1回の線形変換（$y = xW + b$）を行う**単一の**層（部品）です。（$W_q, W_k, W_v, W_o$ はこれです）
* **FFN:** **複数の層（部品）を組み合わせたモジュール**です。`Transformer` のFFNは、以下の3ステップを指します。
    1.  1つ目の「全結合層」（入力次元を拡張する。例: 768 $\rightarrow$ 3072）
    2.  活性化関数（GELUなど。非線形性を導入する）
    3.  2つ目の「全結合層」（出力次元を元に戻す。例: 3072 $\rightarrow$ 768）

### Attention と FFN の役割分担

`Attention` と `FFN` は、明確な役割分担を持っています。

* **Attention (MHA) の役割:**
    * **情報収集と混合 (Mixing)**。Q, K, V を使い、**トークン間**で「文脈的に関連する情報」を集め、Vベクトルを加重平均します。（$Z_i$ は $Z_j$ や $Z_k$ の情報を含む）
* **FFN の役割:**
    * **情報処理と変換 (Processing)**。`Attention` が集めてきた文脈ベクトル（$Z_i$）を、**トークンごとに独立して**受け取り、「処理・計算」します。
    * FFNは `Attention`（加重平均）にはない**非線形性**（GELU）を導入し、より複雑な特徴を抽出・変換します。

`Attention` が「文脈を集める」担当、FFNが「集めた文脈を（非線形に）解釈・処理する」担当、という2つのステップが1セット（＝Transformerブロック）になっています。

> 👩‍🎓 **課題 (ステップ5 考察)**
>
> 1.  上のセルのフォーム機能を使って、`sample_text_ffn` や `token_position` を変えて実行し、`Attention` の出力（$Z$）と `FFN` の出力（$Z'$）が常に**異なる**値になることを確認してください。
> 2.  （考察）`Attention` (MHA) は「情報収集と混合」の役割を担います。では、`FFN` は `Attention` が集めてきた情報をどのように「処理・変換」しているのでしょうか？ ステップ5の解説（特に**入出力の次元数**と**非線形性**）を読んであなたの言葉で説明してください。（自由記述）

## 📚 ステップ6: Transformerブロック (多層化) の意義

ステップ4（MHA）とステップ5（FFN）を1セット（＋Add & Norm）にしたものを「**Transformerブロック**」と呼びます。

1つのブロックだけでは、単純な関係性しか捉えられません。
`Transformer` の性能は、このブロックを何層も（例: `GPT-2` では12層）重ねる（多層化する）ことで発揮されます。

### 階層的な理解

多層化により、AIは**階層的な理解**を行います。

* **1層目 (Block 1):** 「単語＋位置ベクトル」を入力し、`Apple` と `Store` が結びつく（局所的な関係）
* **2層目 (Block 2):** **[1層目の出力（文脈ベクトル）]** を入力とし、`[Apple Store]` と `[buy]` が結びつく（句の関係）
* **3層目 (Block 3):** **[2層目の出力（文脈ベクトル）]** を入力とし、`[buy Apple Store]` と `[I went to]` が結びつく（文全体の意図）
* ...
* **12層目 (Block 12):** 最終的な文脈が出力される

下の層の出力（文脈ベクトル）が、上の層の入力となり、そこで**再び** $W_q, W_k, W_v$ に変換される... という処理が繰り返されます。

次のコードで、層（Layer）の深さによって、`Attention`（Multi-Head）の注目パターンがどのように変化するかを可視化します。

In [99]:
#@title 模式図を表示
from IPython.display import display, HTML

svg_code_string = """<svg width="1200px" viewBox="0 0 1200 500" xmlns="http://www.w3.org/2000/svg">
  <defs>
    <linearGradient id="inputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#4A90E2;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#5BA3F5;stop-opacity:1" />
    </linearGradient>

    <linearGradient id="outputGrad" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:#27AE60;stop-opacity:1" />
      <stop offset="100%" style="stop-color:#52BE80;stop-opacity:1" />
    </linearGradient>

    <marker id="arrowhead" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="#555" />
    </marker>
  </defs>

  <text x="600" y="25" font-size="18" font-weight="bold" text-anchor="middle" fill="#333">
    Stacking Transformer Blocks (Multi-Layer Architecture)
  </text>

  <text x="600" y="50" font-size="13" text-anchor="middle" fill="#666">
    GPT-2では12層のTransformer Blockが積み重なっています
  </text>

  <g id="text_input" transform="translate(30, 150)">
    <rect x="0" y="0" width="60" height="200" fill="#F8F9FA" stroke="#CCC" stroke-width="1" rx="3"/>
    <text x="30" y="20" font-size="12" font-weight="bold" text-anchor="middle" fill="#333">Text</text>

    <text x="30" y="70" font-size="11" text-anchor="middle" fill="#555">"man</text>
    <text x="30" y="120" font-size="11" text-anchor="middle" fill="#555">bites</text>
    <text x="30" y="170" font-size="11" text-anchor="middle" fill="#555">dog"</text>
  </g>

  <path d="M 95 250 L 115 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="embedding" transform="translate(120, 150)">
    <rect x="0" y="0" width="70" height="200" fill="#8E44AD" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Word</text>
    <text x="35" y="35" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Embed</text>

    <line x1="10" y1="70" x2="60" y2="70" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="120" x2="60" y2="120" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="170" x2="60" y2="170" stroke="white" stroke-width="1.5" opacity="0.6"/>
  </g>

  <g id="positional_encoding" transform="translate(120, 60)">
    <rect x="0" y="0" width="70" height="60" fill="#E67E22" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Positional</text>
    <text x="35" y="35" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Encoding</text>
    <text x="35" y="52" font-size="9" text-anchor="middle" fill="white">(PE)</text>
  </g>

  <path d="M 155 125 L 155 145" stroke="#E67E22" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <circle cx="155" cy="135" r="12" fill="white" stroke="#333" stroke-width="1.5"/>
  <text x="155" y="141" font-size="16" font-weight="bold" text-anchor="middle" fill="#333">+</text>

  <path d="M 195 250 L 215 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="input_vec" transform="translate(220, 150)">
    <rect x="0" y="0" width="70" height="200" fill="url(#inputGrad)" stroke="#333" stroke-width="2" rx="3"/>
    <text x="35" y="20" font-size="11" font-weight="bold" text-anchor="middle" fill="white">Input</text>
    <text x="35" y="35" font-size="10" text-anchor="middle" fill="white">Vec</text>

    <line x1="10" y1="70" x2="60" y2="70" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="120" x2="60" y2="120" stroke="white" stroke-width="1.5" opacity="0.6"/>
    <line x1="10" y1="170" x2="60" y2="170" stroke="white" stroke-width="1.5" opacity="0.6"/>
  </g>

  <path d="M 295 250 L 315 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="block1" transform="translate(320, 150)">
    <rect x="0" y="0" width="90" height="200" fill="none" stroke="#2C3E50"
          stroke-width="3" rx="8" stroke-dasharray="8,4" opacity="0.8"/>
    <text x="45" y="30" font-size="12" font-weight="bold" text-anchor="middle" fill="#2C3E50">Block 1</text>
  </g>

  <path d="M 415 250 L 465 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="block2" transform="translate(470, 150)">
    <rect x="0" y="0" width="90" height="200" fill="none" stroke="#2C3E50"
          stroke-width="3" rx="8" stroke-dasharray="8,4" opacity="0.8"/>
    <text x="45" y="30" font-size="12" font-weight="bold" text-anchor="middle" fill="#2C3E50">Block 2</text>
  </g>

  <path d="M 565 250 L 675 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <text x="735" y="260" font-size="24" font-weight="bold" text-anchor="middle" fill="#666">...</text>
  <text x="735" y="285" font-size="11" text-anchor="middle" fill="#666">(Block 3~11)</text>

  <path d="M 795 250 L 875 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="block12" transform="translate(880, 150)">
    <rect x="0" y="0" width="90" height="200" fill="none" stroke="#2C3E50"
          stroke-width="3" rx="8" stroke-dasharray="8,4" opacity="0.8"/>
    <text x="45" y="30" font-size="12" font-weight="bold" text-anchor="middle" fill="#2C3E50">Block 12</text>
  </g>

  <path d="M 975 250 L 1005 250" stroke="#555" stroke-width="2" fill="none" marker-end="url(#arrowhead)"/>

  <g id="lm_head" transform="translate(1010, 170)">
    <rect x="0" y="0" width="90" height="160" fill="#E74C3C" stroke="#333" stroke-width="2" rx="3"/>
    <text x="45" y="30" font-size="12" font-weight="bold" text-anchor="middle" fill="white">LM Head</text>
    <text x="45" y="55" font-size="10" text-anchor="middle" fill="white">(Language</text>
    <text x="45" y="70" font-size="10" text-anchor="middle" fill="white">Modeling Head)</text>

    <text x="45" y="100" font-size="9" text-anchor="middle" fill="white">Next Word</text>
    <text x="45" y="115" font-size="9" text-anchor="middle" fill="white">Prediction</text>

    <text x="45" y="140" font-size="8" text-anchor="middle" fill="white" opacity="0.7">"a" (45%)</text>
    <text x="45" y="152" font-size="8" text-anchor="middle" fill="white" opacity="0.7">"the" (23%)</text>
  </g>

  <g id="explanation">
    <rect x="30" y="400" width="1140" height="80" fill="#F8F9FA" stroke="#DDD" stroke-width="1" rx="5"/>

    <text x="600" y="425" font-size="13" font-weight="bold" text-anchor="middle" fill="#333">
      多層アーキテクチャの重要性
    </text>

    <g id="flow1" transform="translate(50, 445)">
      <circle cx="0" cy="0" r="3" fill="#9B59B6"/>
      <text x="12" y="4" font-size="11" fill="#333">
        各Transformerブロックは前の層の出力を受け取り、より抽象的な表現を学習します
      </text>
    </g>

    <g id="flow2" transform="translate(50, 465)">
      <circle cx="0" cy="0" r="3" fill="#2C3E50"/>
      <text x="12" y="4" font-size="11" fill="#333">
        浅い層（Block 1-2）: 単純なパターン（語順、構文） → 深い層（Block 12）: 複雑な文脈理解（意味、推論）
      </text>
    </g>
  </g>
</svg>
"""

# HTML() を使って表示します
display(HTML(svg_code_string))

In [ ]:
#@title ステップ6: Multi-Layer Attention の可視化 (層ごとの比較)
#@markdown ---
#@markdown ### フォーム
#@markdown Attentionの重みを可視化したい入力文を指定してください。
sample_text_layer = "The quick brown fox jumps over the lazy dog" #@param {type:"string"}
#@markdown 比較したい層（Layer）とヘッド（Head）を選んでください。
layer_index = 1 #@param {type:"slider", min:0, max:11, step:1}
head_index_layer = 0 #@param {type:"slider", min:0, max:11, step:1}
#@markdown ---

# --- GPT-2モデルのロード (ロード済みか確認) ---
if 'model_gpt_with_attentions' not in globals():
    tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
    model_gpt_with_attentions = GPT2Model.from_pretrained("gpt2", output_attentions=True).to(device)
    model_gpt_with_attentions.eval()
    print("GPT-2 モデル (Attention出力付き) をロードしました。")

# --- 入力 ---
inputs = tokenizer_gpt(sample_text_layer, return_tensors="pt").to(device)
tokens = [tokenizer_gpt.decode([token_id]) for token_id in inputs['input_ids'][0]]

# --- モデル実行 (Attentionを取得) ---
with torch.no_grad():
    outputs = model_gpt_with_attentions(**inputs)
    # attentions は12層分のタプル
    attentions_all_layers = outputs.attentions

print(f"\n入力トークン: {tokens}")

# --- 指定された層・ヘッドのAttention重みを取得 ---
try:
    # (Layer, Batch, Head, SeqLen, SeqLen)
    attn_weights = attentions_all_layers[layer_index][0, head_index_layer, :, :].cpu().numpy()

    # --- 可視化 (ヒートマップ) ---
    plt.figure(figsize=(10, 8))
    sns.heatmap(attn_weights,
                annot=True,
                fmt=".2f",
                cmap="viridis",
                xticklabels=tokens,
                yticklabels=tokens)
    plt.title(f"Attention Weights - Layer {layer_index}, Head {head_index_layer}")
    plt.xlabel("Key")
    plt.ylabel("Query")
    plt.show()

    print(f"\n➡️ フォームで 'layer_index' を 0（浅い層）と 11（深い層）で切り替えてみてください。")
    print("   （同じヘッド番号でも）層によって注目パターンが異なることがわかります。")
    print("   浅い層 (Layer 0) は比較的単純なパターン（例: 直前を見る）が多いのに対し、")
    print("   深い層 (Layer 11) は文全体の複雑な関係性を捉えていることがあります。")

except IndexError as e:
    print(f"\nエラー: トークン化に失敗したか、指定したインデックスが不正です。 {e}")
except Exception as e:
    print(f"\nエラーが発生しました: {e}")

> 👩‍🎓 **課題 (ステップ6 考察)**
>
> 1.  上のセルのフォーム機能を使って、`layer_index` を 0（浅い層）と 11（深い層）で切り替えて、同じ `head_index` での注目パターンの違いを観察してください。
> 2.  （考察）なぜ `Transformer` は、`Attention` + `FFN` のブロックを1層だけではなく、12層も重ねるのでしょうか？ 層を重ねる（多層化する）ことの意義について、ステップ6の解説を読んであなたの言葉で説明してください。（自由記述）

## 🎓 ステップ7: LM Head (最終的な予測)

第1課題の復習です。
ここまでのステップ（1～6）を経て、`GPT-2` の**最後のブロック**（12層目）が、**最後の入力トークン**（例: `buy`）に対応する「最終的な文脈ベクトル」（$Z'_{final, last}$）を出力します。

このベクトル（例: 768次元）には、「`I went to the Apple Store to buy`」という文脈がすべて集約されています。

この「最終的な文脈ベクトル」を、最後の全結合層である **`LM Head`** に入力します。

* **`LM Head` の役割:**
    この文脈ベクトル（768次元）を、辞書の総単語数（例: 50257次元）のベクトル（**ロジット**）に変換することです。
* **`Softmax`:**
    このロジットに `Softmax` を適用すると、「次の単語」の確率分布が得られます。

（※ `GPT-2` では、`LM Head` の重み行列は、ステップ1の「単語埋め込み層 (`wte`)」の重み行列と**共有**されています。これはパラメータ効率のためのテクニックです）

以下のコード（第1課題とほぼ同じ）を実行し、これら全てのステップが「次単語予測」に繋がっていることを確認しましょう。

In [ ]:
#@title ステップ7: LM Head による次単語予測 (第1課題の復習)
#@markdown ---
#@markdown ### フォーム
#@markdown 次の単語を予測させたい「入力文（プロンプト）」を指定してください。
prompt_text_final = "I went to the Apple Store to buy" #@param {type:"string"}
#@markdown 表示したい予測候補の数
top_k_value_final = 10 #@param {type:"slider", min:1, max:20, step:1}
#@markdown ---

from transformers import GPT2LMHeadModel # LMHead付きモデルをインポート

# --- LMHead付きGPT-2モデルのロード ---
if 'model_lm_head' not in globals():
    tokenizer_gpt = GPT2Tokenizer.from_pretrained("gpt2")
    model_lm_head = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
    model_lm_head.eval()
    print("GPT-2 (LMHead付き) モデルとトークナイザをロードしました。")
else:
    print("GPT-2 (LMHead付き) モデルはロード済みです。")

# --- 1. トークナイズ ---
inputs_gpt = tokenizer_gpt(prompt_text_final, return_tensors="pt").to(device)
input_ids = inputs_gpt['input_ids']

print(f"入力文: '{prompt_text_final}'")
tokens_display = [tokenizer_gpt.decode([token_id]) for token_id in input_ids[0]]
print(f"  -> トークン: {tokens_display}")

# --- 2. モデルに入力 (全ステップ(1~6)が内部で実行される) ---
with torch.no_grad():
    # outputs.logits には、LM Head の出力（ロジット）が入っている
    outputs_gpt = model_lm_head(**inputs_gpt)

# --- 3. ロジットの取得 ---
logits = outputs_gpt.logits
if logits.shape[1] == 0:
     print("エラー: 入力が空です。")
else:
    # --- 4. 「最後のトークン」のロジットを取得 ---
    # (これが「最終的な文脈ベクトル」が LM Head を通過した結果)
    last_token_logits = logits[0, -1, :]

    # --- 5. Softmax で確率に変換 ---
    probabilities = F.softmax(last_token_logits, dim=-1)

    # --- 6. 確率が高い単語を表示 ---
    top_k_probs, top_k_indices = torch.topk(probabilities, top_k_value_final)

    print(f"\n--- 「{prompt_text_final}」の次にくる単語の予測 (Top {top_k_value_final}) ---")

    results = []
    for i in range(top_k_value_final):
        token_id = top_k_indices[i].item()
        token_str = tokenizer_gpt.decode(token_id)
        prob = top_k_probs[i].item()
        results.append((token_str, prob * 100))

    df_probs = pd.DataFrame(results, columns=["Predicted Word", "Probability (%)"])
    print(df_probs.to_string(index=False))

    # --- 7. 可視化 (英語) ---
    plt.figure(figsize=(10, 5))
    df_probs.set_index("Predicted Word").plot(kind='bar', legend=False)
    plt.title(f"Next word probabilities for '{prompt_text_final}'")
    plt.ylabel("Probability (%)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

> 👩‍🎓 **課題 (ステップ7 考察)**
>
> 1.  上のセルのフォーム機能を使って、`prompt_text_final` を好きな文に変えて実行し、GPTの予測を観察してください。
> 2.  （考察）このセルが予測した「次の単語」は、`Transformer` のどのコンポーネント（部品）によって計算されたものでしょうか？（ヒント：ステップ2～7のすべての部品が関わっています）
> 3.  （考察）第1課題と、この第2課題（QKV, FFN, Multi-Head, PE）を終えて、`Attention` が「文脈を理解する」とは、具体的にどのような計算プロセスであると理解したか、あなたの言葉で説明してください。（自由記述）